In [1]:
import pyspiel

In [2]:
game = pyspiel.load_game("leduc_poker")

In [3]:
state = game.new_initial_state()

In [26]:
print(state.observation_string(0))
print(state.legal_actions())
print(state.information_state_tensor(0))


[Round 1][Player: -1][Pot: 2][Money: 99 99[Private: -10000][Ante: 1 1]
[0, 1, 2, 3, 4, 5]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


IndexError: vector::_M_range_check: __n (which is 0) >= this->size() (which is 0)